# Выбор локации для бурения новой скважины

**Заказчик:** добывающая компания «ГлавРосГосНефть» 

**Задача:** выбор локации, где бурить новую скважину

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализировать возможную прибыль и риски техникой *Bootstrap.*

**Шаги для выбора локации:**

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Условия задачи:**

* Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
* При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
* Бюджет на разработку скважин в регионе — 10 млрд рублей.
* При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
* После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import balanced_accuracy_score, mean_squared_error

pd.options.mode.chained_assignment = None

In [2]:
df1=pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')
df2=pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')
df3=pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv')

In [ ]:
df1.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [ ]:
df1.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [ ]:
df1.duplicated().sum()

0

Были проверены 3 региона на наличие пропусков и анамалий

## Обучение и проверка модели

### Деление на выборки

In [ ]:
features_df1=df1.drop(labels=['product','id'],axis=1)
target_df1=df1['product']

np.random.seed(12345)
features_train_df1, features_valid_df1, target_train_df1, target_valid_df1 = train_test_split(
                                         features_df1, target_df1, test_size=0.25) 

In [ ]:
features_df2=df2.drop(labels=['product','id'],axis=1)
target_df2=df2['product']

np.random.seed(12345)
features_train_df2, features_valid_df2, target_train_df2, target_valid_df2 = train_test_split(
                                         features_df2, target_df2, test_size=0.25) 

In [ ]:
features_df3=df3.drop(labels=['product','id'],axis=1)
target_df3=df3['product']

np.random.seed(12345)
features_train_df3, features_valid_df3, target_train_df3, target_valid_df3 = train_test_split(
                                         features_df3, target_df3, test_size=0.25) 

### Применение нормализации

In [ ]:
scaler = preprocessing.MinMaxScaler()
columns=['f0','f1','f2']

# нормализируем выборки методом MinMaxScaler
scaler.fit(features_train_df1)
features_train_df1_np = scaler.transform(features_train_df1)
features_valid_df1_np = scaler.transform(features_valid_df1)
features_train_df1_norm = pd.DataFrame(features_train_df1_np, columns=columns)
features_valid_df1_norm = pd.DataFrame(features_valid_df1_np, columns=columns)

scaler.fit(features_train_df2)
features_train_df2_np = scaler.transform(features_train_df2)
features_valid_df2_np = scaler.transform(features_valid_df2)
features_train_df2_norm = pd.DataFrame(features_train_df2_np, columns=columns)
features_valid_df2_norm = pd.DataFrame(features_valid_df2_np, columns=columns)

scaler.fit(features_train_df3)
features_train_df3_np = scaler.transform(features_train_df3)
features_valid_df3_np = scaler.transform(features_valid_df3)
features_train_df3_norm = pd.DataFrame(features_train_df3_np, columns=columns)
features_valid_df3_norm = pd.DataFrame(features_valid_df3_np, columns=columns)


### Обучение 

In [ ]:
def train_linear(features_train, target_train, features_valid, target_valid):
    model = LinearRegression() 
    model.fit(features_train,target_train) 
    predictions_valid = model.predict(features_valid) 

    mean =np.mean(predictions_valid).round(3)
    print('Средний запас предсказанного сырья:', mean)
    result = (mean_squared_error(target_valid,predictions_valid)**0.5).round(3) 
    print("RMSE модели линейной регрессии на валидационной выборке:", result)
    return mean, result, predictions_valid

**Обучение на ненормированных данных**

In [ ]:
mean_df1, result_df1, predictions_valid_df1 = train_linear(features_train_df1, target_train_df1,features_valid_df1, target_valid_df1)

Средний запас предсказанного сырья: 92.593
RMSE модели линейной регрессии на валидационной выборке: 37.579


In [ ]:
mean_df2, result_df2, predictions_valid_df2 = train_linear(features_train_df2, target_train_df2,features_valid_df2, target_valid_df2)

Средний запас предсказанного сырья: 68.729
RMSE модели линейной регрессии на валидационной выборке: 0.893


In [ ]:
mean_df3, result_df3, predictions_valid_df3 = train_linear(features_train_df3, target_train_df3, features_valid_df3, target_valid_df3)

Средний запас предсказанного сырья: 94.965
RMSE модели линейной регрессии на валидационной выборке: 40.03


**Обучение на нормированных данных**

In [ ]:
mean_df1_norm, result_df1_norm, predictions_valid_df1_norm = train_linear(features_train_df1_norm, target_train_df1, features_valid_df1_norm, target_valid_df1)

Средний запас предсказанного сырья: 92.593
RMSE модели линейной регрессии на валидационной выборке: 37.579


In [ ]:
mean_df2_norm, result_df2_norm, predictions_valid_df2_norm = train_linear(features_train_df2_norm, target_train_df2, features_valid_df2_norm, target_valid_df2)

Средний запас предсказанного сырья: 68.729
RMSE модели линейной регрессии на валидационной выборке: 0.893


In [ ]:
mean_df3_norm, result_df3_norm, predictions_valid_df3_norm = train_linear(features_train_df3_norm, target_train_df3, features_valid_df3_norm, target_valid_df3)

Средний запас предсказанного сырья: 94.965
RMSE модели линейной регрессии на валидационной выборке: 40.03


In [ ]:
# итоговая таблица
total_table = pd.DataFrame({'Средний запас предсказанного сырья': [mean_df1,mean_df1_norm, mean_df2,mean_df2_norm, mean_df3,mean_df3_norm],
                            'RMSE на валидационной выборке':[result_df1,result_df1_norm, result_df2,result_df2_norm, result_df3,result_df3_norm]},
                           index=['df1','df1_norm','df2','df2_norm','df3','df3_norm'])
total_table

,Средний запас предсказанного сырья,RMSE на валидационной выборке
df1,92.593,37.579
df1_norm,92.593,37.579
df2,68.729,0.893
df2_norm,68.729,0.893
df3,94.965,40.030
df3_norm,94.965,40.030


**Выводы на ненормированных данных:**
* Первый регион - **RMSE 37.58** близко к 0, это хорошо (модель ошибается на 37.58 барр)
* Второй регион - **RMSE 0.89** оочень близко к 0, это очень хорошо
* Третий регион - **RMSE 40.03** близко к 0, это хорошо (модель ошибается на 40.03 барр)

Результаты на нормированных выборках соответствуют результатам на ненормированных данных

## Подготовка к расчёту прибыли

In [ ]:
BUDGET_PER_REGION = 10000000000
STUDIED_POINTS = 500
BEST_POINTS = 200
PRICE_PER_1K_BARR = 450000 # ЦЕНА ЗА 1000 БАРР
MAX_LOSS =  0.025
BOOTSTRAP_SAMPLES = 1000
CONF_INTERVAL = 0.95

<div class="alert alert-block alert-success">
<b>Успех:</b> Хорошо, что в названиях константных переменных использовались только большие буквы!
</div>

In [ ]:
# рассчет выдленного бюджета на исследование одной точки
budget_per_point = BUDGET_PER_REGION / BEST_POINTS
print('Бюджет на исследование одной точки: {:,.2f}'.format(budget_per_point))
# рассчет минимального колличества запаса, которое окупит затраты
min_product = budget_per_point / PRICE_PER_1K_BARR
print('Минимальный объём сырья для безубыточной разработки новой скважины: {:.2f}'.format(min_product))

Бюджет на исследование одной точки: 50,000,000.00
Минимальный объём сырья для безубыточной разработки новой скважины: 111.11


Средний предсказанный запас сырья для каждого региона ниже точки безубыточности.

## Расчёт прибыли и рисков 

In [ ]:
# функция для расчёта прибыли по выбранным скважинам и предсказаниям модели
def revenue(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return selected.sum()

In [ ]:
# функция для рассчета доверительного интервала, средней прибыли, риска убытков, с применением BOOTSTRAP
state = np.random.RandomState(12345)
def risks_profit(target_valid, predictions_valid, features_valid):
    values = []
    for i in range(BOOTSTRAP_SAMPLES):
        target_subsample = target_valid.sample(n=STUDIED_POINTS, replace=True, random_state=state) 
        predictions_valid=pd.Series(predictions_valid,index=features_valid.index)
        probs_subsample = predictions_valid[target_subsample.index] 
        values.append(revenue(target_subsample, probs_subsample, BEST_POINTS)) 
    values = pd.Series(values)
    profit = values*PRICE_PER_1K_BARR-BUDGET_PER_REGION
    profit_mean = (profit.mean()/1000000).round(3)
    
    lower_interval = ((profit.quantile(0.025))/1000000).round(3)
    upper_interval = ((profit.quantile(0.975))/1000000).round(3)

    loss_probability = (profit < 0).mean()
    
    print('Средняя прибыль (млн. руб): {:.2f}'.format(profit_mean))
    print('95% доверительный интервал (млн. руб):{:.2f},{:.2f} '.format(lower_interval, upper_interval) )

    if loss_probability < MAX_LOSS: 
        print('Вероятность убытков равна {}, что меньше чем максимальная допустимая вероятность {}'.format(loss_probability, MAX_LOSS))
    else:
        print('Вероятность убытков равна {}, что больше чем максимальная допустимая вероятность {}'.format(loss_probability, MAX_LOSS))    
    
    return profit_mean, loss_probability, lower_interval, upper_interval

In [ ]:
# показатели для первого региона
risks_profit(target_valid_df1, predictions_valid_df1, features_valid_df1)

Средняя прибыль (млн. руб): 425.94
95% доверительный интервал (млн. руб):-102.09,947.98 
Вероятность убытков равна 0.06, что больше чем максимальная допустимая вероятность 0.025


(425.939, 0.06, -102.09, 947.976)

In [ ]:
# показатели для второго региона
risks_profit(target_valid_df2, predictions_valid_df2, features_valid_df2)

Средняя прибыль (млн. руб): 518.26
95% доверительный интервал (млн. руб):128.12,953.61 
Вероятность убытков равна 0.003, что меньше чем максимальная допустимая вероятность 0.025


(518.259, 0.003, 128.123, 953.613)

In [ ]:
# показатели для третьего региона
risks_profit(target_valid_df3, predictions_valid_df3, features_valid_df3)

Средняя прибыль (млн. руб): 420.19
95% доверительный интервал (млн. руб):-115.85,989.63 
Вероятность убытков равна 0.062, что больше чем максимальная допустимая вероятность 0.025


(420.194, 0.062, -115.853, 989.63)

**Вывод:**

Наибольшая средняя прибыль во втором регионе - **518.26 млн руб.**. Доверительный интервал для 2 региона - от 128 до 954 млн руб. (у единственного границы в положительной зоне)

Вероятность убытков - 0.003, что меньше порогового значения равного 0.025

Таким образом, наиболее предпочтительный регион для разработки скважин - второй